In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("movies.csv")

In [3]:
df.head()

,Unnamed: 0,MovieID,Title,Genres
0,0,1,Toy Story (1995),Animation|Children's|Comedy
1,1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,2,3,Grumpier Old Men (1995),Comedy|Romance
3,3,4,Waiting to Exhale (1995),Comedy|Drama
4,4,5,Father of the Bride Part II (1995),Comedy


In [7]:
user_ratings=pd.read_csv("user_ratings.csv")

FileNotFoundError: [Errno 2] File b'user_rating.csv' does not exist: b'user_rating.csv'

In [ ]:
user_ratings.drop(["Unnamed: 0","user_id"], axis=1, inplace=True)

In [ ]:
user_ratings.head()

In [ ]:
user_ratings=pd.get_dummies(user_ratings, columns=["Gender"], drop_first=True)

In [ ]:
user_ratings.head()

In [ ]:
user_ratings.info()

In [ ]:
!env | grep PYTHONPATH

In [ ]:
import pyspark

spark = (pyspark.sql.SparkSession.builder 
  .master("local[*]")
  .getOrCreate())

In [ ]:
!ls data/

In [ ]:
!file data/user_ratings.csv

In [ ]:
!head data/user_ratings.csv

In [ ]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv('data/user_ratings.csv',
                               inferSchema=True,
                               header=True)

In [ ]:
movie_ratings.printSchema()

In [ ]:
from pyspark.sql.types import (
    ArrayType,
    AtomicType,
    BinaryType,
    BooleanType,
    ByteType,
    CloudPickleSerializer,
    DataType,
    DataTypeSingleton,
    DateConverter,
    DateType,
    DatetimeConverter,
    DecimalType,
    DoubleType,
    FloatType,
    FractionalType,
    IntegerType,
    IntegralType,
    JavaClass,
    LongType,
    MapType,
    NullType,
    NumericType,
    Row,
    ShortType,
    SparkContext,
    StringType,
    StructField,
    StructType,
    TimestampType,
    UserDefinedType,
)

In [ ]:
schema = StructType(
    [
        StructField('user_id', IntegerType()),
        StructField('movie_id', IntegerType()),
        StructField('rating', FloatType()),
        StructField('timestamp', LongType()),
    ]
)

In [ ]:
# # read in the dataset into pyspark DataFrame
# movie_ratings = spark.read.csv('data/ratings.json',
#                                inferSchema=False,
#                                schema=schema,
#                                header=True)

In [ ]:
movie_ratings.persist()

In [ ]:
movie_ratings.show(5)

In [ ]:
# movie_ratings = movie_ratings.drop("timestamp")
# movie_ratings.drop(movie_ratings.col("timestamp"))


df_final = movie_ratings.drop("timestamp")

In [ ]:
movie_ratings.show(15)

In [ ]:
(trainingdata, testdata) = movie_ratings.randomSplit([0.7,0.3], seed=10)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel

als = ALS(
    rank=10,
    maxIter=10,
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
)

In [ ]:
als_model = als.fit(trainingdata)

In [ ]:
predictions = als_model.transform(testdata)

In [ ]:
predictions.persist()

In [ ]:
movie_ratings.show(1)

In [ ]:
user_factors = als_model.userFactors

In [ ]:
user_factors

In [ ]:
item_factors = als_model.itemFactors

In [ ]:
item_factors.show(4)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import train_test_split
from sklearn.model_selection import train_test_split

X = user_ratings.drop(["rating"], axis=1)
y = user_ratings["rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=142, stratify = y)

forest_amount = RandomForestClassifier(n_estimators=30, max_depth= 9)
forest_amount.fit(X_train, y_train)
forest_amount.score(X_train, y_train) # Accuracy of training data
forest_amount.score(X_test, y_test) # Accuracy of test data

# plot_feature_importances(forest_amount) # plot using function above
